In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.dont_write_bytecode
import os
import json
import time
# TOPOLOGIES
from network_topologies.pseudogeant import Pseudogeant
from network_topologies.geant import Geant
from network_topologies.topology import Topology
from network_topologies.topology_preprocessing import preprocess_metadata
# ROUTING ALGORITHMS
from routing_algorithms.dijkstra import dijkstra
from routing_algorithms.dijkstra import calculate_path
from routing_algorithms.dijkstra import set_spt
from routing_algorithms.ear import ear
from routing_algorithms.mora import *
# NETWORK OBJECTS
from utils.network_objects import *
# PA SERVICES
from service_flows.traffic_generator import TrafficGenerator
from itertools import permutations
import random
import numpy as np
from deap import algorithms, base, creator, tools, algorithms
import collections
import matplotlib.pyplot as plt
import import_ipynb
import operator
import pandas as pd

In [3]:
np.random.seed(64)


In [4]:
topo = Geant(routing_method='MORA')
topo.save_topology_info()

Resetting network topology... OK!


In [5]:
# INSTANTIATE TRAFFIC GENERATOR
path = '/Users/federicocolangelo/Documents/Codici/MORA-Multi-Objective-Routing-Algorithm/service_flows/test_traffic/'
set_spt(topo)
tg = TrafficGenerator(interval=60, topology=topo, path=path, faults = 1)
time.sleep(1)


******* GENERATE_FLOWS -> ITERATION 1 OUT OF 12 *******
++++ BEGINNING OF ITERATION @ 04/15/2020, 17:04:25 ++++

#### NODE FAILURES PHASE... execution time = 8.106231689453125e-06
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 9.012222290039062e-05
#### GENERATE NEW FLOWS PHASE... execution time = 

/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMultiObj' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


100221 0.08330439062500039 0.0009610446129246003 2 2 0 0 - -
107.89768600463867
++++ END OF ITERATION @ 04/15/2020, 17:06:13 ++++
******* GENERATE_FLOWS -> ITERATION 1 OUT OF 12 ELAPSED TIME = 107.90857696533203 *******
******* INTERVAL EXCEEDED BY 47.90861415863037 SECONDS *******
--------------------------------------------------------------------------

******* GENERATE_FLOWS -> ITERATION 2 OUT OF 12 *******
++++ BEGINNING OF ITERATION @ 04/15/2020, 17:06:13 ++++

#### NODE FAILURES PHASE... execution time = 1.0013580322265625e-05
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 6.699562072753906e-05
#### GENERATE NEW FLOWS PHASE... execution time = 108642 0.33868326122500036 0.0039446251570652265 2 2 0 0 - -
113.33128190040588
++++ END OF ITERATION @ 04/15/2020, 17:08:06 ++++
******* GENERATE_FLOWS -> ITERATION 2 OUT OF 12 ELAPSED TIME = 113.33189105987549 *******
******* INTERVAL EXCEEDED BY 53.33215308189392 SECONDS *******
------------------------------------------------

/Users/federicocolangelo/Documents/Codici/MORA-Multi-Objective-Routing-Algorithm/service_flows/traffic_generator.py:331: ComplexWarning: Casting complex values to real discards the imaginary part
  print(int(self.topo.get_power_consumption()), max_rel, mean_rel,\
/Users/federicocolangelo/Documents/Codici/MORA-Multi-Objective-Routing-Algorithm/service_flows/traffic_generator.py:334: ComplexWarning: Casting complex values to real discards the imaginary part
  row = pd.Series([self.topo.routing_method, int(self.topo.get_power_consumption()), max_rel, mean_rel,\
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Users/federicocolangelo/opt/anaconda3/envs/experiments/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/federicocolangelo/Documents/Codici/MORA-Multi-Objective-Routi

29111 0 0.0 2 2 0 0 - -
34.36150908470154
#### GENERATE NEW FLOWS PHASE... execution time = 

In [ ]:
test = tg.flows['ATEEassured']

In [17]:
l = topo.get_one_link('ROHU')

In [19]:
l.get_power_consumption(-1)

(359.99838196601127-0.0019021130325903072j)

In [22]:
mu = 1e-03
alpha = 1.4
n_l = 1
mu * ((-1/n_l)**alpha)

(-0.0003090169943749476-0.0009510565162951536j)

In [25]:
(-1)**1.4

(-0.30901699437494756-0.9510565162951535j)

In [11]:
for link in topo.links:
    print(link.id, link.power_consumption_MORA)

LNLT 367.0676944823639
HUSI 360.0
CHFR 370.9782409504611
IEUK 429.2614109374357
UKPT 430.05798345382806
FRES 0.0
GCAT 0.0
ATCZ 416.6262036583108
DEDU 460.2548088703212
UIFR 362.7880827395436
UIUK 370.97824095046064
LVET 0.0
PLDE 386.1642710615376
CZHU 0
GRIT 371.0006148879615
ATGC 555.5593926753577
SIHU 360.0
GRGC (360-1.0002638421964765e-20j)
BENL 478.78088953583875
DUNL 440.2337483070271
ITSI 360.0
ETEE 0.0
PRPT 469.54469970409986
ATIT 360.0
ROHU (360-3.6077013775481106e-20j)
NLBE 423.6839006789466
HUAT 360.018400369501
PLLT (360-8.297090828036678e-21j)
NLUK 422.54048749751337
IEIR 0.0
ATHU 363.61216173437003
CZAT 366.50003878325157
HUHR 0.0
ATSK 360.0
ESFR 0.0
DEPL 426.86113017937026
DUDE 364.50004182129845
UIIR 360.43029191333926
HRHU 360.0
ITAT 495.7517864411543
EEET 367.08747380719814
HURO 0.0
DUEE 367.0874738071981
EEDU (360-2.0622750822474075e-22j)
DECZ 378.66163018573457
BGRO (360-2.0271730219279606e-20j)
ESPR 0.0
FRCH 361.0636979964776
DENL 407.2162351256994
NLDE 577.48454483

In [ ]:
get_path = get_optimize_route(topo)

In [ ]:

topo.meta_heuristic = 1

In [ ]:
%timeit get_path(test)

In [64]:
from utils.network_objects import Flow
from routing_algorithms.mora import mutate_path, crossover_one_point
indi = []
flow = Flow(test)
for i in range(5000):
    indi.append(generate_individual(list, flow.starting_node, flow.ending_node, topo))
indi[5]

['AT',
 'HR',
 'HU',
 'CZ',
 'DE',
 'CH',
 'ES',
 'FR',
 'UI',
 'UK',
 'BE',
 'NL',
 'DU',
 'EE']

In [66]:
%timeit mutate_path(indi[5], topo, list)

48.5 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
--------------------------------------------------------------------------

******* GENERATE_FLOWS -> ITERATION 2 OUT OF 12 *******
++++ BEGINNING OF ITERATION @ 04/15/2020, 12:10:38 ++++

#### NODE FAILURES PHASE... execution time = 1.0967254638671875e-05
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 3.981590270996094e-05
#### GENERATE NEW FLOWS PHASE... execution time = 

In [72]:
%timeit mutate_path_faster(indi[5], topo, list)

662 µs ± 64.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
--------------------------------------------------------------------------

******* GENERATE_FLOWS -> ITERATION 8 OUT OF 12 *******
++++ BEGINNING OF ITERATION @ 04/15/2020, 12:16:38 ++++

#### NODE FAILURES PHASE... execution time = 9.059906005859375e-06
#### DISRUPTED FLOWS REROUTING PHASE... execution time = 3.910064697265625e-05
#### GENERATE NEW FLOWS PHASE... execution time = 18.27043604850769
++++ END OF ITERATION @ 04/15/2020, 12:16:56 ++++
******* GENERATE_FLOWS -> ITERATION 8 OUT OF 12 ELAPSED TIME = 18.286648988723755 *******


In [74]:
topology=topo
flow_obj = Flow(test)
# number of weights in the tuple -> number of objective functions
creator.create("FitnessMultiObj", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0,)) 
creator.create("Individual", list, fitness=creator.FitnessMultiObj)
toolbox = base.Toolbox()
toolbox.register("individual", generate_individual, creator.Individual, flow_obj.starting_node, flow_obj.ending_node, topology)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover_one_point, topology=topology, ind_class=creator.Individual, toolbox=toolbox)
toolbox.register("select", tools.selNSGA2)
toolbox.register("mutate", mutate_path, topology=topology, indi_class=creator.Individual)

evaluate_individual = get_evaluate_individual(topology, test)
toolbox.register("evaluate", evaluate_individual)
evaluate_SLA = get_evaluate_SLA(flow_obj.SLA, topology, evaluate_individual)
penalty = get_penalty(flow_obj.SLA, topology, evaluate_individual) 
toolbox.decorate("evaluate", tools.DeltaPenality(evaluate_SLA, 20.0, penalty))

In [87]:
from routing_algorithms.mora import get_faster_optimize_route
faster_optimize_flow = get_faster_optimize_route(topo)

In [90]:
# Pop = 25
%timeit faster_optimize_flow(test)

329 ms ± 68.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%timeit faster_optimize_flow(test)

671 ms ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%timeit optimize_flow(test)

5.12 s ± 940 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit crossover_one_point(indi[5], indi[6], topo, list, toolbox)

In [ ]:
%timeit evaluate_individual(indi[5])

In [3]:
cols = ['Routing algorithm', 'Power consumption [W]', 'Reliability score (Max)',\
                    'Reliability score (Mean)', 'Mean latency (premium) [ms]',\
                    'Mean latency (assured) [ms]', 'Premium SLA violations', 'Assured SLA violations',
                    'Mean Premium SLA violations' , 'Mean Assured SLA violations']
df_o = pd.DataFrame(columns=cols)
df_o.to_csv('tezt.csv', mode='w', header=True)


In [58]:
topo.init_MORA()

In [132]:
def enumerate_paths(s, d, max_hops, current_prefix = [], pts = []):
    current_prefix.append(s)
    N = topo.get_valid_links(current_prefix[-1])
    N = [x for x in N if x not in current_prefix]
    for n in N:
        if d == n:
            new = current_prefix.copy()
            new.append(d)
            pts.append(new)
            continue
        if len(current_prefix) <= max_hops-1:
            pts = enumerate_paths(n, d, max_hops, current_prefix, pts)
    current_prefix.pop(-1)
    return pts         